In [7]:
# from __future__ import annotations
# from dataclasses import dataclass, field
# 
# import numpy as np
# import pandas as pd
# import typing as tp
# import jijmodeling as jm
# import pathlib
# import uuid
# 
# @dataclass
# class DataNode:
#     data: tp.Any
#     name: str | None = None
#     operator: FunctionNode | None = None
# 
# 
# class FunctionNode:
#     name: str | None = None
# 
#     def __init__(
#         self,
#     ) -> None:
#         self.inputs = []
# 
#     def apply(self, inputs: list[DataNode], **kwargs: tp.Any) -> DataNode:
#         self.inputs += inputs
#         return self.operate(inputs, **kwargs)
# 
#     def operate(self, inputs: list[DataNode], **kwargs: tp.Any) -> DataNode:
#         raise NotImplementedError


In [ ]:
# DEFAULT_RESULT_DIR = "./.jb_results"
# 
# 
# @dataclass
# class ID(DataNode):
#     data: str | uuid.UUID = field(default_factory=uuid.uuid4)
# 
#     def __post_init__(self):
#         self.data = str(self.data)
# 
# 
# @dataclass
# class Date(DataNode):
#     data: str | pd.Timestamp = field(default_factory=pd.Timestamp.now)
#     name: str = "timestamp"
# 
#     def __post_init__(self):
#         if isinstance(self.data, str):
#             self.data = pd.Timestamp(self.data)
# 
# 
# class Min(FunctionNode):
#     name = "min"
# 
#     def operate(self, inputs: list[Array]) -> DataNode:
#         data = inputs[0].data.min()
#         name = inputs[0].name + f"_{self.name}" if inputs[0].name else None
#         node = DataNode(data=data, name=name, operator=self)
#         return node
# 
# 
# class Max(FunctionNode):
#     name = "max"
# 
#     def operate(self, inputs: list[Array]) -> DataNode:
#         data = inputs[0].data.max()
#         name = inputs[0].name + f"_{self.name}" if inputs[0].name else None
#         node = DataNode(data=data, name=name, operator=self)
#         return node
# 
# 
# class Mean(FunctionNode):
#     name = "mean"
# 
#     def operate(self, inputs: list[Array]) -> DataNode:
#         data = inputs[0].data.mean()
#         name = inputs[0].name + f"_{self.name}" if inputs[0].name else None
#         node = DataNode(data=data, name=name, operator=self)
#         return node
# 
# 
# class Std(FunctionNode):
#     name = "std"
# 
#     def operate(self, inputs: list[Array]) -> DataNode:
#         data = inputs[0].data.std()
#         name = inputs[0].name + f"_{self.name}" if inputs[0].name else None
#         node = DataNode(data=data, name=name, operator=self)
#         return node
# 
# 
# @dataclass
# class Array(DataNode):
#     data: np.ndarray
# 
#     def min(self) -> DataNode:
#         return Min().apply([self])
# 
#     def max(self) -> DataNode:
#         return Max().apply([self])
# 
#     def mean(self) -> DataNode:
#         return Mean().apply([self])
# 
#     def std(self) -> DataNode:
#         return Std().apply([self])
# 
# 
# @dataclass
# class Energy(Array):
#     name: str = "energy"
# 
# 
# @dataclass
# class Objective(Array):
#     name: str = "objective"
# 
# 
# @dataclass
# class ConstraintViolation(Array):
#     def __post_init__(self):
#         if self.name is None:
#             raise NameError("Attribute 'name' is None. Please set a name.")
# 
# 
# @dataclass
# class SampleSet(DataNode):
#     data: jm.SampleSet
# 
# 
# class RecordFactory(FunctionNode):
#     name = "record"
# 
#     def operate(self, inputs: list[DataNode], name: str | None = None) -> Record:
#         data = pd.Series({node.name: node.data for node in inputs})
#         return Record(data, name=name, operator=self)
# 
# 
# class TableFactory(FunctionNode):
#     name = "table"
# 
#     def operate(self, inputs: list[Record], name: str | None = None) -> Table:
#         data = pd.DataFrame({node.name: node.data for node in inputs})
#         return Table(data, name=name, operator=self)
# 
# 
# class ArtifactFactory(FunctionNode):
#     name = "artifact"
# 
#     def operate(self, inputs: list[Record], name: str | None = None) -> Artifact:
#         data = {node.name: node.data.to_dict() for node in inputs}
#         return Artifact(data, name=name, operator=self)
# 
# 
# @dataclass
# class Record(DataNode):
#     data: pd.Series = field(default_factory=lambda: pd.Series(dtype="object"))
# 
# 
# @dataclass
# class Table(DataNode):
#     data: pd.DataFrame = field(default_factory=pd.DataFrame)
# 
#     def append(self, record: Record, axis: tp.Literal[0, 1] = 0) -> Table:
#         table = TableFactory().apply([record])
#         return Concat().apply([self, table], axis=axis)
# 
# 
# @dataclass
# class Artifact(DataNode):
#     data: dict = field(default_factory=dict)
# 
# 
# class Concat(FunctionNode):
#     name = "concat"
# 
#     def operate(
#         self, inputs: list[Table] | list[Artifact], name=None, axis: tp.Literal[0, 1] = 0
#     ) -> Table | Artifact:
#         dtype = type(inputs[0])
#         if not all([isinstance(node, dtype) for node in inputs]):
#             raise TypeError(
#                 "Type of elements of 'inputs' must be unified with either 'Table' or 'Artifact'."
#             )
# 
#         if isinstance(inputs[0], Artifact):
#             data = {node.name: node.data for node in inputs}
#             return Artifact(data=data, name=name, operator=self)
#         elif isinstance(inputs[0], Table):
#             data = pd.concat(
#                 [node.data for node in inputs if isinstance(node, Table)], axis=axis
#             )
#             return Table(data=data, name=name, operator=self)
#         else:
#             raise TypeError(f"'{inputs[0].__class__.__name__}' type is not supported.")
# 
# 
# @dataclass
# class Experiment(DataNode):
#     data: tuple = ()
# 
#     def __post_init__(self):
#         if not self.data:
#             self.data = (Table(), Artifact())

In [91]:
from __future__ import annotations
from dataclasses import dataclass, field

import numpy as np
import pandas as pd
import typing as tp
import jijmodeling as jm
import pathlib
import uuid

T_in = tp.TypeVar("T_in", bound="DataNode", covariant=True)
T_out = tp.TypeVar("T_out", bound="DataNode", covariant=True)
T_f = tp.TypeVar("T_f", bound="FunctionNode", covariant=True)
T_c = tp.TypeVar("T_c", "Artifact", "Table")

@dataclass
class DataNode(tp.Generic[T_f]):
    data: tp.Any
    name: str | None = None
    operator: T_f | None = None


class FunctionNode(tp.Generic[T_in, T_out]):
    name: str | None = None

    def __init__(
        self,
    ) -> None:
        self.inputs = []

    def apply(self, inputs: list[T_in], **kwargs: tp.Any) -> T_out:
        self.inputs += inputs
        return self.operate(inputs, **kwargs)

    def operate(self, inputs: list[T_in], **kwargs: tp.Any) -> T_out:
        raise NotImplementedError


DEFAULT_RESULT_DIR = "./.jb_results"

@dataclass
class ID(DataNode):
    data: str = field(default_factory=lambda: str(uuid.uuid4()))
    
    def __post_init__(self):
        self.data = str(self.data)
    

@dataclass
class Date(DataNode):
    data: str | pd.Timestamp = field(default_factory=pd.Timestamp.now)
    name: str = "timestamp"
    
    def __post_init__(self):
        if isinstance(self.data, str):
            self.data = pd.Timestamp(self.data)


class Min(FunctionNode["Array", "DataNode[Min]"]):
    name = "min"
    
    def operate(self, inputs: list[Array]) -> DataNode[Min]: 
        data = inputs[0].data.min()
        name = inputs[0].name + f"_{self.name}" if inputs[0].name else None
        node = DataNode(data=data, name=name, operator=self)
        return node
  
class Max(FunctionNode["Array", "DataNode[Max]"]):
    name = "max"
    
    def operate(self, inputs: list[Array]) -> DataNode[Max]:
        data = inputs[0].data.max()
        name = inputs[0].name + f"_{self.name}" if inputs[0].name else None
        node = DataNode(data=data, name=name, operator=self)
        return node



class Mean(FunctionNode["Array", "DataNode[Mean]"]):
    name = "mean"
    
    def operate(self, inputs: list[Array]) -> DataNode:
        data = inputs[0].data.mean()
        name = inputs[0].name + f"_{self.name}" if inputs[0].name else None
        node = DataNode(data=data, name=name, operator=self)
        return node
    
class Std(FunctionNode["Array", "DataNode[Std]"]):
    name = "std"
    
    def operate(self, inputs: list[Array]) -> DataNode[Std]:
        data = inputs[0].data.std()
        name = inputs[0].name + f"_{self.name}" if inputs[0].name else None
        node = DataNode(data=data, name=name, operator=self)
        return node


@dataclass
class Array(DataNode):
    data: np.ndarray
    
    def min(self) -> DataNode:
        return Min().apply([self])
    
    def max(self) -> DataNode:
        return Max().apply([self])
    
    def mean(self) -> DataNode:
        return Mean().apply([self])
    
    def std(self) -> DataNode:
        return Std().apply([self])
    
    
@dataclass
class Energy(Array):
    name: str = "energy"

@dataclass
class Objective(Array):
    name: str = "objective"

@dataclass
class ConstraintViolation(Array):
    def __post_init__(self):
        if self.name is None:
            raise NameError("Attribute 'name' is None. Please set a name.")

@dataclass
class SampleSet(DataNode):
    data: jm.SampleSet


class RecordFactory(FunctionNode["T_in", "Record"]):
    name = "record"
    
    def operate(self, inputs: list[T_in], name: str | None = None) -> Record:
        data = pd.Series({node.name: node.data for node in inputs})
        return Record(data, name=name, operator=self)
    
    
class TableFactory(FunctionNode["Record", "Table"]):
    name = "table"
    
    def operate(self, inputs: list[Record], name: str | None = None) -> Table:
        data = pd.DataFrame({node.name: node.data for node in inputs})
        return Table(data, name=name, operator=self)
    
class ArtifactFactory(FunctionNode["Record", "Artifact"]):
    name = "artifact"
    
    def operate(self, inputs: list[Record], name: str | None = None) -> Artifact:
        data = {node.name: node.data.to_dict() for node in inputs}
        return Artifact(data, name=name, operator=self)
    

@dataclass
class Record(DataNode):
    data: pd.Series = field(default_factory=lambda: pd.Series(dtype="object"))


@dataclass
class Table(DataNode):
    data: pd.DataFrame = field(default_factory=pd.DataFrame)
    
    def append(self, record: Record, axis:tp.Literal[0, 1]=0) -> Table:
        table = TableFactory().apply([record], name=self.name)
        return Concat().apply([self, table], axis=axis)

@dataclass
class Artifact(DataNode):
    data: dict = field(default_factory=dict)
    
    def append(self, record: Record) -> Artifact:
        artifact = ArtifactFactory().apply([record], name=self.name)
        return Concat().apply([self, artifact])
    

class Concat(FunctionNode[T_c, T_c]):
    name = "concat"
    
    def operate(self, inputs: list[T_c], name: str| None = None, axis: tp.Literal[0, 1]=0) -> T_c:
        dtype = type(inputs[0])
        if not all([isinstance(node, dtype) for node in inputs]):
            raise TypeError("Type of elements of 'inputs' must be unified with either 'Table' or 'Artifact'.")
        
        if isinstance(inputs[0], Artifact):
            data = {}
            for node in inputs:
                if node.name in data:
                    data[node.name].update(node.data)
                else:
                    data[node.name] = node.data
            return Artifact(data=data, name=name, operator=self)
        elif isinstance(inputs[0], Table):
            data = pd.concat([node.data for node in inputs if isinstance(node, Table)], axis=axis)
            return Table(data=data, name=name, operator=self)
        else:
            raise TypeError(f"'{inputs[0].__class__.__name__}' type is not supported.")
        
@dataclass
class Experiment(DataNode):
    data: tuple = ()
    
    def __post_init__(self):
        if not self.data:
            self.data = (Table(), Artifact())    

In [ ]:







    


# class Experiment(FunctionNode):
#     def __init__(self, name: str | None = None, autosave=True, save_dir: str = DEFAULT_RESULT_DIR) -> None:
#         children = [Table(), Artifact()]
#         if name is None:
#             name = str(ID().data)
#         super().__init__(name, children)
#         
#         self.autosave = autosave
#         self.save_dir = pathlib.Path(save_dir)
#     
#     @property
#     def table(self) -> Table:
#         children = self._narrow_children_type()
#         return children[0]
#     
#     @property
#     def artifact(self) -> Artifact:
#         children = self._narrow_children_type()
#         return children[1]
#     
#     def __enter__(self) -> Experiment:
#         p = self.save_dir / str(self.name)
#         (p / "table").mkdir(parents=True)
#         (p / "artifact").mkdir(parents=True)
#         return self
#         
#     def __exit__(self, exception_type, exception_value, traceback) -> None:
#         if self.autosave:
#             pass
#         
#     def append(self, record: Record) -> None:
#         self.table.append(record)
#         self.artifact.append(record)
        

In [ ]:
T1 = tp.TypeVar("T1")
T2 = tp.TypeVar("T2", bound=float)

class A(tp.Generic[T1, T2]):
    def f(self, x: T1) -> T2:
        raise NotImplementedError
    

class B(A[int, float]):
    def f(self, x: int) -> float:
        return float(x)
    
b = B()
b.f(10)


In [6]:
array = Array(np.arange(10))
isinstance(array, DataNode)

True

In [ ]:
from torch.utils.data import Dataset

In [3]:
i = ID()
print(i.data)
print(i.children)
print(i.name)

1dc5a427-386b-4ad8-8f24-08dedbb531df
None
None


In [40]:
e0 = Experiment()
e0.data[0].data["a"] = [0, 1, 2]
e0.data

e0 = Experiment()
e0.data

(Table(data=   a
 0  0
 1  1
 2  2, name=None, operator=None),
 Artifact(data={}, name=None, operator=None))

In [93]:
a = 1
a.__class__.__name__

'int'

In [49]:
pd.Series(dtype="object")

Series([], dtype: object)

In [25]:
d0 = DataNode(0, name="d0")
d1 = DataNode(1, name="d1")
d2 = DataNode(2, name="d2")

r0 = RecordFactory().operate([d0, d1], name="new r0")
r1 = RecordFactory().operate([d0, d2], name="new r1")
r0.data

d0    0
d1    1
dtype: int64

In [14]:
t0 = Table(pd.DataFrame([0, 1], index=["d0", "d1"], columns=["a"]))
t1 = Table(pd.DataFrame([2, 3], index=["d0", "d1"], columns=["b"]))
tc = Concat().operate([t0, t1], axis=1)

tc.data

,a,b
d0,0,2
d1,1,3


In [15]:
t2 = t1.append(r1, axis=1)
t2.data


,b,new r1
d0,2.0,0.0
d1,3.0,NaN
d2,NaN,2.0


In [61]:
a = ArtifactFactory().apply([r0])
a

Artifact(data={'new r0': {'d0': 0, 'd1': 1}}, name=None, operator=<__main__.ArtifactFactory object at 0x7fa8d20a4820>)

In [90]:
a0 = Artifact({"a": 0, "b": 1}, name=ID().data)
a1 = Artifact({"c": 0, "d": 1}, name=ID().data)

a0.append(r1).data

{'10b92481-fd1b-4d0e-a47b-9056e15f3f7c': {'a': 0,
  'b': 1,
  'new r1': {'d0': 0, 'd2': 2}}}

In [81]:
r = Record()
r.data["a"] = 1
r.data

r1 = Record()
r1.data

r.data

a    1
dtype: int64

In [82]:
x = {}
x1 = {"a": 1}
# x.update(x1)
x.get("b", {}).update(x1)
# x = {}.update(x1)

In [42]:
if ():
    print("a")
else:
    print("b")

b


In [239]:
import numpy as np

array = Array(np.arange(10))
print(array.data)
mi = array.min()
if mi.operator is not None:
    print(mi.operator.inputs[0])
    
print(array.min())
print(array.max())

[0 1 2 3 4 5 6 7 8 9]
Array(data=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), name=None, operator=<__main__.FunctionNode object at 0x7f1e0b754760>, children=None)
DataNode(data=0, name=None, operator=<__main__.Min object at 0x7f1e0b754d90>)
DataNode(data=9, name=None, operator=<__main__.FunctionNode object at 0x7f1e0b754610>)


In [29]:
d0 = DataNode(pd.Series([0, 1, 2]), name="a")
d1 = DataNode(pd.Series([2, 3, 4]), name="b")

x = {d0.name: d0.data, d1.name: d1.data}
pd.DataFrame(x)

,a,b
0,0,2
1,1,3
2,2,4


In [189]:
@dataclass
class A:
    x: int
    
    def __post_init__(self):
        self.y = None
        
    def f(self):
        self.y += 1
        
a = A(1)
a.y




In [5]:
import jijzept as jz
import jijmodeling as jm

sampler = jz.JijSASampler(config="/home/d/.jijzept/config.toml")

problem = jm.Problem("sample")
x = jm.Binary("x", 5)
problem += x[:]
problem += jm.Constraint("onehot", x[:] == 1)

res = sampler.sample_model(problem, {})

In [49]:
res.feasible()

SampleSet(record=Record(solution={'x': [(([1],), [1], (5,))]}, num_occurrences=[1]), evaluation=Evaluation(energy=[0.0], objective=[1.0], constraint_violations={'onehot': [0.0]}, penalty={}), measuring_time=MeasuringTime(solve=SolvingTime(preprocess=None, solve=0.009699344635009766, postprocess=None), system=SystemTime(post_problem_and_instance_data=0.8710920810699463, request_queue=0.4920237064361572, fetch_problem_and_instance_data=None, fetch_result=2.3001277446746826, deserialize_solution=7.462501525878906e-05), total=3.6673190593719482))

In [6]:
id0 = ID(name="benchmark_id")
energy = Energy(np.array(res.evaluation.energy))
objective = Objective(np.array(res.evaluation.objective))
if res.evaluation.constraint_violations is not None:
    const_name, const_values  = list(res.evaluation.constraint_violations.items())[0]
    constraint_violation = ConstraintViolation(np.array(const_values), const_name)
else:
    constraint_violation = ConstraintViolation()

print(energy)
print(objective)
print(constraint_violation)

Energy(data=array([0.]), name='energy', children=None)
Objective(data=array([0.]), name='objective', children=None)
ConstraintViolation(data=array([1.]), name='onehot', children=None)


In [7]:
record = Record(
    name=id0.data,
    children=[id0, energy, objective, constraint_violation]
)
record.to_dict()

{'benchmark_id': 'ebe7ecf4-c3f4-4818-98c8-e1541f089f77',
 'energy': array([0.]),
 'objective': array([0.]),
 'onehot': array([1.])}

In [9]:
record.to_series()

benchmark_id    ebe7ecf4-c3f4-4818-98c8-e1541f089f77
energy                                         [0.0]
objective                                      [0.0]
onehot                                         [1.0]
dtype: object

In [8]:
table = Table(children=[record])
table.to_dataframe()

,benchmark_id,energy,objective,onehot
0,ebe7ecf4-c3f4-4818-98c8-e1541f089f77,[0.0],[0.0],[1.0]


In [10]:
table.append(record)
table.to_dataframe()

,benchmark_id,energy,objective,onehot
0,db27355a-04ef-4e53-81df-719d6fe7fc52,[0.0],[0.0],[1.0]
1,db27355a-04ef-4e53-81df-719d6fe7fc52,[0.0],[0.0],[1.0]


In [10]:
artifact = Artifact(children=[record])
artifact.to_dict()

{'ebe7ecf4-c3f4-4818-98c8-e1541f089f77': {'benchmark_id': 'ebe7ecf4-c3f4-4818-98c8-e1541f089f77',
  'energy': array([0.]),
  'objective': array([0.]),
  'onehot': array([1.])}}

In [11]:
id1 = ID(name="benchmark_id")
record2 = Record(
    name=id1.data,
    children=[id1, energy, objective, constraint_violation]
)
artifact.append(record2)
artifact.to_dict()

{'ebe7ecf4-c3f4-4818-98c8-e1541f089f77': {'benchmark_id': 'ebe7ecf4-c3f4-4818-98c8-e1541f089f77',
  'energy': array([0.]),
  'objective': array([0.]),
  'onehot': array([1.])},
 '2198eae5-1c80-476c-9d7d-53b7f7d1cfb8': {'benchmark_id': '2198eae5-1c80-476c-9d7d-53b7f7d1cfb8',
  'energy': array([0.]),
  'objective': array([0.]),
  'onehot': array([1.])}}

In [12]:
print(id0)
print(id1)

ID(data='ebe7ecf4-c3f4-4818-98c8-e1541f089f77', name='benchmark_id', children=None)
ID(data='2198eae5-1c80-476c-9d7d-53b7f7d1cfb8', name='benchmark_id', children=None)


In [100]:
experiment = Experiment()

In [101]:
experiment.append(record)
experiment.append(record2)


In [102]:
df = experiment.table.to_dataframe()
df

,benchmark_id,energy,objective,onehot,timestamp
0,ebe7ecf4-c3f4-4818-98c8-e1541f089f77,[0.0],[0.0],[1.0],2023-01-10 08:21:21.162533
1,2198eae5-1c80-476c-9d7d-53b7f7d1cfb8,[0.0],[0.0],[1.0],2023-01-10 08:21:21.162621


In [81]:
experiment.table.children

In [59]:
Date()

Date(data=Timestamp('2023-01-10 07:38:21.498293'), name='timestamp', children=None)

In [73]:
ID()

ID(data='870be0e2-d49f-409d-abbe-aea4b54a95ce', name=None, children=None)

In [25]:
experiment.table.children

[]

In [19]:
ID().data

'5ba2df6b-d1ca-471e-825f-b29eb053f986'

In [23]:
p = pathlib.Path("./a")
(p / "b").mkdir(parents=True)

In [25]:
pd.Timestamp("2022-01-01")

Timestamp('2022-01-01 00:00:00')

In [107]:
class ArrayVisitor:
    def min(self, array: Array):
        return float(array.data.min())
    

array = Array(np.arange(10))
array.accept_min(ArrayVisitor())


0.0

In [204]:
from chainer.variable import Variable, VariableNode
from chainer.function_node import FunctionNode
from chainer.function import FunctionAdapter

a = Variable(np.array(10.0), name="a")
x = Variable(np.array(1.0), name="x")
y = Variable(np.array(2.0), name="y")
z = a * x + y

z.backward(retain_grad=True)

print(z)
print(z.node)
print("=======")
print(z.node.creator.inputs)
print(z.node.creator.inputs[0].data)
print(z.node.creator.inputs[0].name)
print(z.node.creator.inputs[1].data)
print(z.node.creator.inputs[1].name)
print("@@@@@@@@@@")
print(z.node.creator.inputs[0].creator.inputs[0].data)
print(z.node.creator.inputs[0].creator.inputs[0].name)
print(z.node.creator.inputs[0].creator.inputs[1].data)
print(z.node.creator.inputs[0].creator.inputs[1].name)
print(z.node.creator.inputs[1].data)
print(z.node.creator.inputs[1].name)
print("++++++++++")
print(z.node.creator.inputs[0].creator.inputs[0].creator)
print(z.node.creator.inputs[0].creator.inputs[1].creator)
print()



variable(12.)
(<chainer.variable.VariableNode object at 0x7f1e0b8f5070>, <chainer.variable.VariableNode object at 0x7f1e28e34340>)
None
None
None
y
@@@@@@@@@@
10.0
a
1.0
x
None
y
++++++++++
None
None



<bound method Variable.mean of variable(12.)>

In [ ]:
from chainer.functions import add

In [39]:
s1 = pd.Series([1, 2, 3], index=["a", "b", "c"])
s2 = pd.Series([1, 2, 3], index=["a", "b", "c"])

In [43]:
pd.concat([s1], axis=1).T

,a,b,c
0,1,2,3


ModuleNotFoundError: No module named 'jijbench'

In [ ]:
# Tn = tp.TypeVar("Tn")

# class FunctionNode(tp.Generic[Tn]):
#     def __init__(
#         self, 
#         name: str | None = None, 
#         children: list[Tn] | None = None
#     ) -> None:
#         self._name = name``
#         self._children = children
#         
#     @property
#     def name(self) -> str | None:
#         return self._name
#     
#     @property
#     def children(self) -> list[Tn] | None:
#         return self._children
#     
#     def _narrow_children_type(self) -> list[Tn]:
#         if self.children is None:
#             raise ValueError(
#                 "Value of attribute 'children' is None, which means this node has no children. Therefore, it cannot operate node futher more."
#             )
#         return self.children
#     
#     def append(self, node: Tn) -> None:
#         children = self._narrow_children_type()
#         children.append(node)
#     
    
            
# Tn = tp.TypeVar("Tn", bound="BaseNode")
# 
# class BaseNode(tp.Generic[Tn]):
#     pass
# 
# @dataclass
# class DataNode(BaseNode[Tn]):
#     data: tp.Any = None
#     name: str | None = None
#     children: list[Tn] | None = None
#     
#     
# class FunctionNode(BaseNode[Tn]):
#     def __init__(
#         self, 
#         name: str | None = None, 
#         children: list[Tn] | None = None
#     ) -> None:
#         self.name = name
#         if children is None:
#             self.children = []
#         else:
#             self.children = children
# 